In [1]:
# Imports 
import numpy as np
import gdsfactory as gf
from functools import partial
import math 
from gdsfactory.read.from_np import from_image
from gdsfactory.path import Path, extrude

gf.CONF.cell_layout_cache = False
#c.plot()

2025-07-24 09:54:05.660 | INFO     | kfactory.kcell:show:3535 - klive v0.3.3: Reloaded file 'C:\Users\Cristina\AppData\Local\Temp\368760230.gds'


# PARAMETERS

In [2]:
# Parameters definition

layer_metal = (4,0)
layer_box = (100,0)

die_w = 10000
die_L = 5000

dicing_edge = 125 

"""
layer_shallow = (1,0)
layer_deep = (2,0)
layer_prot_deep = (11,0)
layer_prot_shallow = (21,0)


num_per_side = 37
spacing = 127
die_side = 'left'
offset_x = dicing_edge/2
offset_y = dicing_edge

w_wg_gen = 1.0+0.2
w_wg_w1 = 2.5+0.2
w_wg_w2 = 4.0+0.2

w_wg_md_gen = 1.0+0.2
w_wg_md_w1 = 2.5+0.2
w_wg_md_w2 = 4.0+0.2

w_wg_shal_gen = 1.0+0.2
w_wg_shal_w1 = 2.5+0.2
w_wg_shal_w2 = 4.0+0.2

w_slab = 15.0
w_prot_deep = 15.0

w_taper_io = 0.75+0.2

h_wg_gen = 0.3
h_wg_mini = 0.15

w_metal = 3.5
"""

"\nlayer_shallow = (1,0)\nlayer_deep = (2,0)\nlayer_prot_deep = (11,0)\nlayer_prot_shallow = (21,0)\n\n\nnum_per_side = 37\nspacing = 127\ndie_side = 'left'\noffset_x = dicing_edge/2\noffset_y = dicing_edge\n\nw_wg_gen = 1.0+0.2\nw_wg_w1 = 2.5+0.2\nw_wg_w2 = 4.0+0.2\n\nw_wg_md_gen = 1.0+0.2\nw_wg_md_w1 = 2.5+0.2\nw_wg_md_w2 = 4.0+0.2\n\nw_wg_shal_gen = 1.0+0.2\nw_wg_shal_w1 = 2.5+0.2\nw_wg_shal_w2 = 4.0+0.2\n\nw_slab = 15.0\nw_prot_deep = 15.0\n\nw_taper_io = 0.75+0.2\n\nh_wg_gen = 0.3\nh_wg_mini = 0.15\n\nw_metal = 3.5\n"

# PDK CELLS

In [3]:
@gf.cell
def die(dieL = 5000, dieW = 10000, border = 250, layer_box = (101,0)):
    box = gf.Component()
    obox = box.add_ref(gf.components.rectangle(size=(dieW,dieL),layer=layer_box))
    ibox = box.add_ref(gf.components.rectangle(size=(dieW-border*2,dieL-border*2),layer=layer_box)).dmovex(border).dmovey(border)
    box = gf.boolean(A=obox, B=ibox, operation="A-B", layer=layer_box)
    box.add_port(name="block@org", center=[border,border], width=1, orientation=0, layer=layer_box)
    return box

In [4]:
wafer = gf.Component(name='w')

c1 = gf.Component(name='c1')

box7 = c1.add_ref(die(dieW = die_w, dieL = die_L, border = dicing_edge, layer_box=layer_box ))

# RESISTENCIAS

In [5]:
import gdsfactory as gf
from gdsfactory.component import Component
from gdsfactory.components import rectangle, taper  # función que genera un rectángulo de tamaño (x, y) en una capa específica. Genera un taper.

def huella_resistencia(
    layer=layer_metal,             # Capa GDS (layer, datatype)
    Xc=100,                   # Centro X del conjunto
    Yc=2,                     # Centro Y del conjunto
    tam_Rx=10,                # Largo (X) de la resistencia
    tam_Ry=1,                 # Ancho (Y) de la resistencia
    padx=5,                   # Largo (X) de los pads
    pady=5,                   # Ancho (Y) de los pads
    length_taper=0            # Largo (X) del taper 
) -> Component:
    c = gf.Component()

    # Largo total del dispositivo de manera horizontal
    total_length = 2 * padx + tam_Rx + 2 * length_taper # Suma de todos los elementos en el eje X: pad izquierdo + taper + resistencia + taper + pad derecho.
    x0 = Xc - total_length / 2  # inicio en X --> Para ubicar x0 en el borde izquierdo de todo el dispositivo
    y0 = Yc                     # centro vertical

    # Pad izquierdo
    pad1 = rectangle(size=(padx, pady), layer=layer) 
    ref1 = c.add_ref(pad1)
    ref1.move((x0, y0 - pady / 2)) # x -> el pad izquierdo justo en el borde izquierdo del conjunto. y -> el pad tiene un tamaño vertical pady, y para centrarlo en y0 restamos la mitad de su alto. Así, el centro vertical del pad queda en y0.

    # Taper izquierdo (de ancho pady -> ancho tam_Ry)
    taper1 = taper(
        length=length_taper,
        width1=pady,
        width2=tam_Ry,
        layer=layer
    )
    ref_taper1 = c.add_ref(taper1)
    ref_taper1.move((x0+ padx, y0))

    # Resistencia central
    res = rectangle(size=(tam_Rx, tam_Ry), layer=layer)
    ref2 = c.add_ref(res)
    ref2.move((x0 + padx + length_taper, y0 - tam_Ry / 2)) # x -> Se avanza en el eje X. x0 es el inicio + padx salimos del pad izquierdo + length_taper para incluir la longitud del taper.  y -> y = y0 - tam_Ry/2: centra la resistencia verticalmente en y0, igual que el pad.

    # Taper derecho (de ancho tam_Ry -> ancho pady)
    taper2 = taper(
        length=length_taper,
        width1=tam_Ry,
        width2=pady,
        layer=layer
    )
    ref_taper2 = c.add_ref(taper2)
    ref_taper2.move((x0 + padx + length_taper + tam_Rx, y0))

    # Pad derecho
    pad2 = rectangle(size=(padx, pady), layer=layer)
    ref3 = c.add_ref(pad2)
    ref3.move((x0 + padx + length_taper + tam_Rx + length_taper, y0 - pady / 2)) # x -> Empezamos en el borde izquierdo (x0) y avanzamos: padx (pad izquierdo) + length_taper (taper izquierdo) + tam_Rx (resistencia) + length_taper (taper derecho) -- Así ubicamos el pad derecho al final de la resistencia y tapers.  y -> y = y0 - pady/2: centramos verticalmente el pad derecho igual que el izquierdo.

    return c




In [6]:
res_msh1_up = huella_resistencia(
    layer=layer_metal,
    Xc=2146.43069,
    Yc=4322.49817,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh1_up_ = c1.add_ref(res_msh1_up)

res_msh1_down = huella_resistencia(
    layer=layer_metal,
    Xc=2146.43069,
    Yc=3742.49707,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh1_down_ = c1.add_ref(res_msh1_down)

res_m1_up = huella_resistencia(
    layer=layer_metal,
    Xc=1873.43208,
    Yc=3254.49761,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m1_up_ = c1.add_ref(res_m1_up)

res_m1_down_m2_up = huella_resistencia(
    layer=layer_metal,
    Xc=1873.43208,
    Yc=2668.39900,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m1_down_m2_up_ = c1.add_ref(res_m1_down_m2_up)

res_m2_down = huella_resistencia(
    layer=layer_metal,
    Xc=1873.43208,
    Yc=2171.49751,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m2_down_ = c1.add_ref(res_m2_down)

res_msh4_up = huella_resistencia(
    layer=layer_metal,
    Xc=2146.43011,
    Yc=1274.49845,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh4_up_ = c1.add_ref(res_msh4_up)

res_msh4_down = huella_resistencia(
    layer=layer_metal,
    Xc=2146.43011,
    Yc=694.49975,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh4_down_ = c1.add_ref(res_msh4_down)

res_msh2_down = huella_resistencia(
    layer=layer_metal,
    Xc=5323.43119,
    Yc=4322.49947,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh2_down_ = c1.add_ref(res_msh2_down)

res_msh3_up = huella_resistencia(
    layer=layer_metal,
    Xc=5323.43119,
    Yc=3742.49930,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh3_up_ = c1.add_ref(res_msh3_up)

res_m3_up = huella_resistencia(
    layer=layer_metal,
    Xc=5083.43032,
    Yc=2668.39952,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m3_up_ = c1.add_ref(res_m3_up)

res_m3_down = huella_resistencia(
    layer=layer_metal,
    Xc=5083.43032,
    Yc=2171.49937,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m3_down_ = c1.add_ref(res_m3_down)

res_msh5_down = huella_resistencia(
    layer=layer_metal,
    Xc=5323.43625,
    Yc=1274.49968,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh5_down_ = c1.add_ref(res_msh5_down)

res_msh6_up = huella_resistencia(
    layer=layer_metal,
    Xc=5323.43625,
    Yc=694.49947,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_msh6_up_ = c1.add_ref(res_msh6_up)

res_m4_up = huella_resistencia(
    layer=layer_metal,
    Xc=8293.430965,
    Yc=2668.39955,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m4_up_ = c1.add_ref(res_m4_up)

res_m4_down = huella_resistencia(
    layer=layer_metal,
    Xc=8293.430965,
    Yc=2171.49913,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m4_down_ = c1.add_ref(res_m4_down)

res_m5_down = huella_resistencia(
    layer=layer_metal,
    Xc=8293.430965,
    Yc=1585.39933,
    tam_Rx=1630,
    tam_Ry=7,
    padx=110,
    pady=110,
    length_taper=100
)

res_m5_down_ = c1.add_ref(res_m5_down)

#res_msh1_up.plot()

In [7]:
## Oblea 

#wafer.add_ref(c)
wafer.add_ref(c1)
wafer.show()

wafer.write_gds("Resistencias_Ruben.gds")

WindowsPath('Resistencias_Ruben.gds')